In [1]:
import pandas as pd
import numpy as np
import time

데이터 불러오기

In [2]:
with open('2016_filtered_review_part.txt', encoding='utf-8') as f:
    docs = [doc.strip().split('\t\t') for doc in f]
    docs = [(doc[1], int(doc[2])) for doc in docs if len(doc) == 3]
    # To read the second and third column info from each row
    texts, scores = zip(*docs)
    # 둘을 분리해서 별도의 list 변수로 저장

평점 정보를 이용해서 종속변수 레이블링 하기

In [3]:
filtered_texts = []
filtered_labels = []

for text, score in zip(texts, scores):
    if 4 < score < 8:
        continue
        
    # 평점 기준으로 문서에 label을 부여
    # 1 ~ 4 -> 부정, 0
    # 8 ~ 10 -> 긍정, 1
    filtered_texts.append(text)
    filtered_labels.append(1 if score >= 8 else 0)

In [4]:
# 불균형 문제
print('전체 영화평의 수: {}'.format(len(filtered_labels)))
print('긍정 영화평의 수: {}'.format(sum(filtered_labels)))
print('긍정 영화평의 비율: {0:.2f}'.format(sum(filtered_labels)/len(filtered_labels)))

전체 영화평의 수: 19513
긍정 영화평의 수: 17890
긍정 영화평의 비율: 0.92


In [5]:
# To split the data into training and test datasets
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.1, random_state=42)

In [6]:
train_texts[0]

'영화 괜찮 재미있'

# 1. L1 규제화 사용해 보기

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
lr_tf_l1 = LogisticRegression(C=0.1, penalty='l1', solver='saga', max_iter=10000) 

### 1.1 TF 정보 사용해 보기

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer() 
tf_train_features = tf_vectorizer.fit_transform(train_texts) 
tf_test_features = tf_vectorizer.transform(test_texts)

In [10]:
start = time.time()
lr_tf_l1.fit(tf_train_features, train_labels) # 학습
end = time.time()
elapsed = end - start 
print('\nLearning time: {0:.3f} seconds'.format(elapsed))


Learning time: 0.918 seconds


In [11]:
pred_labels_tf_l1 = lr_tf_l1.predict(tf_test_features) # 예측

In [12]:
from sklearn.metrics import accuracy_score

In [13]:
accuracy_score(test_labels, pred_labels_tf_l1)

0.9369877049180327

긍정과 부정의 역할을 하는 단어 출력해 보기

In [14]:
# 단어 사전 만들기
words_dict = {}
for word, index in tf_vectorizer.vocabulary_.items():
    words_dict[index]=word

In [15]:
# Get coefficients of the model
coefficients = lr_tf_l1.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# 학습에 사용된 각 단어마다의 coefficient (즉 weight) 값이 존재
# coefficient값이 큰 순으로 정렬 'reverse=True'


K=30
# print top K positive words
print("긍정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[:K]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))
# print top K negative words
print("\n부정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[-K:]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))

긍정 단어 상위 30 개
재밌 (1.523)
재미있 (1.371)
최고 (1.266)
역시 (0.924)
영상미 (0.768)
도르마무 (0.662)
아이 (0.640)
다음 (0.584)
추천 (0.517)
영상 (0.478)
마블 (0.431)
스트레인 (0.349)
연기 (0.329)
약간 (0.317)
닥터스 (0.235)
기대 (0.232)
효과 (0.213)
멋지 (0.177)
망토 (0.175)
베네딕트 (0.157)
베니 (0.157)
아이맥스 (0.143)
생각 (0.114)
조금 (0.102)
괜찮 (0.089)
닌자 (0.079)
독특 (0.074)
거북이 (0.037)
시각 (0.033)
19금 (0.000)

부정 단어 상위 30 개
나오 (-0.271)
이해 (-0.274)
졸리 (-0.284)
너무 (-0.335)
영화 (-0.345)
처음 (-0.363)
기분 (-0.379)
개연 (-0.435)
진심 (-0.437)
스토리 (-0.524)
음악 (-0.525)
노래 (-0.560)
핵노잼 (-0.567)
그냥 (-0.601)
초딩 (-0.637)
정신 (-0.662)
손예진 (-0.692)
내용 (-0.741)
평점 (-0.814)
유치 (-0.850)
쓰레기 (-0.931)
실망 (-0.970)
감독 (-1.291)
낭비 (-1.338)
별로 (-1.364)
짜증 (-1.411)
알바 (-1.861)
재미없 (-2.509)
최악 (-2.946)
노잼 (-3.038)


### 1.2 TF-IDF 정보 사용해 보기

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer() 
tfidf_train_features = tfidf_vectorizer.fit_transform(train_texts) 
tfidf_test_features = tfidf_vectorizer.transform(test_texts)

In [17]:
lr_tfidf_l1 = LogisticRegression(C=0.1, penalty='l1', solver='saga', max_iter=10000) 
lr_tfidf_l1.fit(tfidf_train_features, train_labels) # 학습
pred_labels_tfidf_l1 = lr_tfidf_l1.predict(tfidf_test_features)

In [18]:
accuracy_score(test_labels, pred_labels_tfidf_l1)

0.9344262295081968

상위 긍, 부정 단어 출력해 보기

In [19]:
coefficients = lr_tfidf_l1.coef_.tolist()
sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)

K=30
# print top K positive words
print("긍정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[:K]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))
# print top K negative words
print("부정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[-K:]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))

긍정 단어 상위 30 개
재밌 (3.786)
재미있 (2.545)
최고 (1.708)
마블 (1.559)
도르마무 (1.029)
역시 (1.026)
영상미 (0.636)
아이 (0.326)
기대 (0.262)
영상 (0.167)
19금 (0.000)
1으 (0.000)
2가 (0.000)
2관 (0.000)
2디 (0.000)
2화 (0.000)
3d (0.000)
3디 (0.000)
3베니마블 (0.000)
4d (0.000)
4디 (0.000)
d강추 (0.000)
oo노잼 (0.000)
ㄱ저냥 (0.000)
ㄹ혜 (0.000)
ㅅ흡 (0.000)
ㅈ노잼 (0.000)
ㅈ루 (0.000)
ㅋㅋ왤 (0.000)
ㅋㅋ재밋게 (0.000)
부정 단어 상위 30 개
히트 (0.000)
히트다히트 (0.000)
히트작 (0.000)
히피 (0.000)
히헿 (0.000)
히히 (0.000)
히히히헤헤헤히힣 (0.000)
힌트 (0.000)
힐로 (0.000)
힐링 (0.000)
힘내 (0.000)
힘들 (0.000)
힘쓰 (0.000)
힘없이 (0.000)
힙합 (0.000)
너무 (-0.007)
짜증 (-0.217)
음악 (-0.282)
스토리 (-0.573)
그냥 (-0.700)
영화 (-1.035)
내용 (-1.234)
손예진 (-1.573)
별로 (-2.301)
평점 (-2.924)
알바 (-3.396)
노잼 (-3.700)
감독 (-3.805)
재미없 (-4.846)
최악 (-6.287)


# L2 규제화 사용해 보기

### TF 정보 사용하기

In [20]:
lr_tf_l2 = LogisticRegression(C=0.1, penalty='l2', solver='saga', max_iter=10000)

In [21]:
start = time.time()
lr_tf_l2.fit(tf_train_features, train_labels) # 학습
end = time.time()
elapsed = end - start 
print('\nLearning time: {0:.3f} seconds'.format(elapsed))


Learning time: 0.498 seconds


In [22]:
pred_labels_tf_l2 = lr_tf_l2.predict(tf_test_features) # 예측

In [23]:
accuracy_score(test_labels, pred_labels_tf_l2)

0.9369877049180327

In [24]:
# Get coefficients of the model
coefficients = lr_tf_l2.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# 학습에 사용된 각 단어마다의 coefficient (즉 weight) 값이 존재
# coefficient값이 큰 순으로 정렬 'reverse=True'


K=30
# print top K positive words
print("긍정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[:K]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))
# print top K negative words
print("\n부정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[-K:]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))

긍정 단어 상위 30 개
재밌 (1.447)
재미있 (1.258)
최고 (1.132)
역시 (0.909)
영상미 (0.765)
아이 (0.746)
도르마무 (0.733)
다음 (0.723)
추천 (0.695)
약간 (0.671)
망토 (0.574)
영상 (0.560)
독특 (0.558)
아이맥스 (0.536)
베니 (0.534)
스트레인 (0.524)
재밋 (0.503)
멋지 (0.500)
효과 (0.499)
닥터스 (0.491)
한국 (0.488)
호강 (0.486)
조금 (0.459)
그리 (0.456)
재미나 (0.446)
마블 (0.444)
연기 (0.443)
시각 (0.439)
아주 (0.423)
만족 (0.415)

부정 단어 상위 30 개
비추 (-0.538)
그냥 (-0.542)
차라리 (-0.572)
개연 (-0.577)
스토리 (-0.579)
노래 (-0.584)
그나마 (-0.627)
막장 (-0.641)
보다 (-0.652)
손예진 (-0.652)
진심 (-0.659)
짬뽕 (-0.667)
졸리 (-0.710)
정신 (-0.743)
내용 (-0.749)
개노잼 (-0.758)
초딩 (-0.780)
평점 (-0.791)
핵노잼 (-0.825)
쓰레기 (-0.860)
유치 (-0.870)
실망 (-0.931)
짜증 (-1.009)
낭비 (-1.040)
감독 (-1.106)
별로 (-1.294)
알바 (-1.422)
노잼 (-1.822)
재미없 (-1.926)
최악 (-1.933)


### TF-IDF 정보 사용하기

In [25]:
lr_tfidf_l2 = LogisticRegression(C=0.1, penalty='l2', solver='saga', max_iter=10000) 
lr_tfidf_l2.fit(tfidf_train_features, train_labels) # 학습
pred_labels_tfidf_l2 = lr_tfidf_l2.predict(tfidf_test_features)

In [26]:
accuracy_score(test_labels, pred_labels_tfidf_l2)

0.930327868852459

In [27]:
# Get coefficients of the model
coefficients = lr_tfidf_l2.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# 학습에 사용된 각 단어마다의 coefficient (즉 weight) 값이 존재
# coefficient값이 큰 순으로 정렬 'reverse=True'


K=30
# print top K positive words
print("긍정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[:K]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))
# print top K negative words
print("\n부정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[-K:]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))

긍정 단어 상위 30 개
재밌 (2.306)
재미있 (1.680)
마블 (1.265)
최고 (1.254)
역시 (1.077)
영상미 (0.912)
도르마무 (0.859)
아이 (0.854)
영상 (0.710)
추천 (0.671)
다음 (0.651)
기대 (0.650)
아이맥스 (0.496)
망토 (0.476)
베네딕트 (0.470)
멋지 (0.457)
약간 (0.451)
닥터스 (0.443)
베니 (0.443)
쿠키 (0.436)
볼거리 (0.432)
효과 (0.428)
트레인 (0.423)
스트레인 (0.416)
거래 (0.405)
닥터 (0.394)
시각 (0.380)
호강 (0.379)
완전 (0.377)
재밋 (0.370)

부정 단어 상위 30 개
기분 (-0.540)
비밀 (-0.550)
초딩 (-0.589)
수준 (-0.591)
개노잼 (-0.591)
개연 (-0.602)
이해 (-0.612)
정신 (-0.626)
노래 (-0.646)
진심 (-0.668)
보다 (-0.687)
실망 (-0.717)
유치 (-0.731)
핵노잼 (-0.758)
그냥 (-0.799)
스토리 (-0.813)
낭비 (-0.817)
쓰레기 (-0.843)
음악 (-0.864)
짜증 (-0.926)
영화 (-0.973)
내용 (-1.071)
손예진 (-1.232)
알바 (-1.533)
별로 (-1.551)
감독 (-1.632)
노잼 (-1.637)
평점 (-1.822)
최악 (-2.245)
재미없 (-2.273)
